# SGAN

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical

Using TensorFlow backend.


## Generator

In [3]:
def build_generator(latent_dim):

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(1, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

# Discriminator

In [4]:
def build_discriminator(img_shape, num_classes):

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())

    model.summary()

    img = Input(shape=img_shape)

    features = model(img)
    valid = Dense(1, activation="sigmoid")(features)
    label = Dense(num_classes + 1, activation="softmax")(features)

    return Model(img, [valid, label])

## helper

In [5]:
def sample_images(G, latent_dim, epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = G.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [6]:
def save(model, model_name):
    model_path = "saved_model/%s.json" % model_name
    weights_path = "saved_model/%s_weights.hdf5" % model_name
    options = {"file_arch": model_path,
                "file_weight": weights_path}
    json_string = model.to_json()
    open(options['file_arch'], 'w').write(json_string)
    model.save_weights(options['file_weight'])

In [7]:
def save_model(G, D, combined):
    save(G, "mnist_sgan_generator")
    save(D, "mnist_sgan_discriminator")
    save(combined, "mnist_sgan_adversarial")

## Train

In [8]:
def train(G, D, combined,
          num_classes, latent_dim,
          epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, y_train), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    y_train = y_train.reshape(-1, 1)

    # Class weights:
    # To balance the difference in occurences of digit class labels.
    # 50% of labels that the discriminator trains on are 'fake'.
    # Weight = 1 / frequency
    half_batch = batch_size // 2
    cw1 = {0: 1, 1: 1}
    cw2 = {i: num_classes / half_batch for i in range(num_classes)}
    cw2[num_classes] = 1 / half_batch

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Sample noise and generate a batch of new images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = G.predict(noise)

        # One-hot encoding of labels
        labels = to_categorical(y_train[idx], num_classes=num_classes + 1)
        fake_labels = to_categorical(np.full((batch_size, 1), num_classes), num_classes=num_classes + 1)

        # Train the discriminator
        d_loss_real = D.train_on_batch(imgs, [valid, labels], class_weight=[cw1, cw2])
        d_loss_fake = D.train_on_batch(gen_imgs, [fake, fake_labels], class_weight=[cw1, cw2])
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


        # ---------------------
        #  Train Generator
        # ---------------------

        g_loss = combined.train_on_batch(noise, valid, class_weight=[cw1, cw2])

        # Plot the progress
        print ("%d [D loss: %f, acc: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0 or epoch == epochs - 1:
            sample_images(G, latent_dim, epoch)


## main()

In [11]:
for d in ['images']:
    if not os.path.exists(d):
        os.makedirs(d)

In [12]:
# initialization

In [13]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
num_classes = 10
latent_dim = 100

In [14]:
# create optimizer
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [16]:
# Build and compile the discriminator
D = build_discriminator(img_shape, num_classes)
D.compile(loss=['binary_crossentropy', 'categorical_crossentropy'],
          loss_weights=[0.5, 0.5],
          optimizer=optimizer,
          metrics=['accuracy']
          )

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)         

In [17]:
# Build the generator
G = build_generator(latent_dim)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 128)      

In [18]:
# The generator takes noise as input and generates imgs
noise = Input(shape=(100,))
img = G(noise)

In [19]:
# For the combined model we will only train the generator
D.trainable = False

In [20]:
# The valid takes generated images as input and determines validity
valid, _ = D(img)

In [21]:
# The combined model  (stacked generator and discriminator)
# Trains generator to fool discriminator
combined = Model(noise, valid)
combined.compile(loss=['binary_crossentropy'], optimizer=optimizer)

In [ ]:
# epochs=20000
epochs=2000
train(G, D, combined,
      num_classes, latent_dim,
      epochs=epochs,
      batch_size=32, sample_interval=50)

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.962485, acc: 46.88%, op_acc: 12.50%] [G loss: 0.307589]


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.842761, acc: 51.56%, op_acc: 6.25%] [G loss: 0.434881]
2 [D loss: 0.672172, acc: 79.69%, op_acc: 14.06%] [G loss: 0.575161]
3 [D loss: 0.557908, acc: 100.00%, op_acc: 12.50%] [G loss: 0.578829]
4 [D loss: 0.499317, acc: 100.00%, op_acc: 23.44%] [G loss: 0.686973]
5 [D loss: 0.437283, acc: 100.00%, op_acc: 37.50%] [G loss: 0.596043]
6 [D loss: 0.435212, acc: 98.44%, op_acc: 46.88%] [G loss: 0.562973]
7 [D loss: 0.540383, acc: 89.06%, op_acc: 34.38%] [G loss: 0.833444]
8 [D loss: 0.523059, acc: 82.81%, op_acc: 28.12%] [G loss: 1.060906]
9 [D loss: 0.586833, acc: 76.56%, op_acc: 25.00%] [G loss: 1.283327]
10 [D loss: 0.543046, acc: 79.69%, op_acc: 31.25%] [G loss: 1.563692]
11 [D loss: 0.614470, acc: 70.31%, op_acc: 29.69%] [G loss: 1.464273]
12 [D loss: 0.581043, acc: 79.69%, op_acc: 15.62%] [G loss: 1.326742]
13 [D loss: 0.510187, acc: 84.38%, op_acc: 28.12%] [G loss: 1.334987]
14 [D loss: 0.576156, acc: 85.94%, op_acc: 31.25%] [G loss: 1.032913]
15 [D loss: 0.516727, acc: 

118 [D loss: 0.636286, acc: 39.06%, op_acc: 50.00%] [G loss: 1.242940]
119 [D loss: 0.678222, acc: 39.06%, op_acc: 40.62%] [G loss: 1.422840]
120 [D loss: 0.634135, acc: 46.88%, op_acc: 45.31%] [G loss: 1.611099]
121 [D loss: 0.662601, acc: 46.88%, op_acc: 48.44%] [G loss: 1.380791]
122 [D loss: 0.636283, acc: 48.44%, op_acc: 42.19%] [G loss: 1.319250]
123 [D loss: 0.714793, acc: 42.19%, op_acc: 37.50%] [G loss: 1.306468]
124 [D loss: 0.667188, acc: 48.44%, op_acc: 34.38%] [G loss: 1.280994]
125 [D loss: 0.722268, acc: 45.31%, op_acc: 43.75%] [G loss: 1.184548]
126 [D loss: 0.590395, acc: 53.12%, op_acc: 35.94%] [G loss: 1.298957]
127 [D loss: 0.574914, acc: 48.44%, op_acc: 56.25%] [G loss: 1.443805]
128 [D loss: 0.572099, acc: 40.62%, op_acc: 46.88%] [G loss: 0.981745]
129 [D loss: 0.659817, acc: 34.38%, op_acc: 46.88%] [G loss: 1.162366]
130 [D loss: 0.691855, acc: 42.19%, op_acc: 42.19%] [G loss: 1.232838]
131 [D loss: 0.577897, acc: 45.31%, op_acc: 45.31%] [G loss: 1.419766]
132 [D

234 [D loss: 0.479889, acc: 51.56%, op_acc: 53.12%] [G loss: 1.297603]
235 [D loss: 0.502652, acc: 50.00%, op_acc: 56.25%] [G loss: 1.247050]
236 [D loss: 0.485431, acc: 62.50%, op_acc: 42.19%] [G loss: 1.328890]
237 [D loss: 0.514294, acc: 51.56%, op_acc: 53.12%] [G loss: 1.222086]
238 [D loss: 0.480768, acc: 64.06%, op_acc: 50.00%] [G loss: 1.514277]
239 [D loss: 0.524882, acc: 56.25%, op_acc: 50.00%] [G loss: 1.168857]
240 [D loss: 0.529448, acc: 51.56%, op_acc: 37.50%] [G loss: 1.403154]
241 [D loss: 0.484373, acc: 65.62%, op_acc: 48.44%] [G loss: 1.409695]
242 [D loss: 0.582908, acc: 46.88%, op_acc: 43.75%] [G loss: 1.120604]
243 [D loss: 0.488423, acc: 57.81%, op_acc: 50.00%] [G loss: 1.293254]
244 [D loss: 0.555186, acc: 46.88%, op_acc: 54.69%] [G loss: 1.124138]
245 [D loss: 0.542650, acc: 56.25%, op_acc: 43.75%] [G loss: 1.082058]
246 [D loss: 0.533609, acc: 54.69%, op_acc: 48.44%] [G loss: 1.085073]
247 [D loss: 0.435694, acc: 70.31%, op_acc: 50.00%] [G loss: 1.251329]
248 [D